In [ ]:
import datetime
import os
from collections import defaultdict

import dateutil.parser
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from benthicnet.utils import sanitize_filename, sanitize_filename_series
from IPython.display import display
from tqdm.auto import tqdm

from pangaea_downloader.tools import checker

In [ ]:
dirname = "../query-outputs_2022-01-01"

## Load datasets and check distribution of column frequencies

In [ ]:
def find_url_column(df):

    clean_cols = [
        col.lower().replace(" ", "").replace("-", "").replace("_", "").replace(".", "")
        for col in df.columns
    ]
    # Ordered list of priorities
    # Exclude url meta/ref/source which are not links to images
    candidates = [
        "urlimage",
        "urlraw",
        "urlfile",
        "url",
        "urlgraphic",
        "urlthumb",
        "urlthumbnail",
        "image",
        "imagery",
    ]
    for candidate in candidates:
        if candidate not in clean_cols:
            continue
        col = df.columns[clean_cols.index(candidate)]
        if any(df[col].apply(checker.is_url)):
            return col

In [ ]:
column_count = defaultdict(lambda: 0)
column_examples = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

verbose = False

for fname in tqdm(os.listdir(dirname)):
    ds_id = os.path.splitext(fname)[0]
    df = pd.read_csv(os.path.join(dirname, fname))
    n_total += 1

    if not checker.has_url_col(df):
        continue

    url_col = find_url_column(df)

    if not url_col:
        if verbose:
            print(f"No url column for {fname} with columns\n{df.columns}")
        files_without_url.append(fname)
        continue

    n_valid += 1
    for col in df.columns:
        col = col.lower().strip()
        column_count[col] += 1
        column_examples[col].append(fname)
    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Examining columns to find out what their contents are

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["resolution"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
urls = []
n_datasets = 0
for fname in column_examples["seagr cov"]:
    df = pd.read_csv(os.path.join(dirname, fname))
    url_column = find_url_column(df)
    if url_column:
        urls.append(df[url_column])
        n_datasets += 1

In [ ]:
len(pd.concat(urls).unique())

In [ ]:
n_datasets

In [ ]:
# Load up largest dataset, containing long cruises
df = pd.read_csv(os.path.join(dirname, "882349.csv"))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(-df["Depth water"], label="Depth water")
plt.plot(df["Elevation"], label="Elevation")
plt.legend()
plt.show()
plt.figure(figsize=(12, 8))
plt.plot(df["Depth water"] + df["Elevation"])
plt.show()

In [ ]:
len(column_examples["seagr cov"])

In [ ]:
len(set(column_examples["seagr cov"]))

In [ ]:
coverage_columns = [k for k in column_examples.keys() if k.endswith(" cov")]

In [ ]:
coverage_datasets = set()
for k in coverage_columns:
    if k == "ice cov":
        continue
    print()
    print(k)
    print(len(coverage_datasets))
    coverage_datasets.update(column_examples[k])
    print(len(coverage_datasets))

In [ ]:
len(coverage_datasets)

In [ ]:
urls = []
n_datasets = 0
for fname in coverage_datasets:
    df = pd.read_csv(os.path.join(dirname, fname))
    url_column = find_url_column(df)
    if url_column:
        urls.append(df[url_column])
        n_datasets += 1

In [ ]:
len(pd.concat(urls).unique())

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["ice cov"][1]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["ph"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["elevation"][-1]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["doi"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, files_with_repeat_urls[3]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[-1])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["latitude south"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

In [ ]:
df = pd.read_csv(os.path.join(dirname, column_examples["date"][0]))
display(df)
print(df.columns)
url_column = find_url_column(df)
print(df[url_column].iloc[0])
print(df["dataset_title"].iloc[0])

## Implement dataset cleaning functions

In [ ]:
TAXONOMY_RANKS = [
    ["Kingdom", "Regnum"],
    ["Phylum", "Division"],
    ["Ordo", "Order"],
    ["Familia", "Family"],
    ["Genus"],
    ["Species"],
]


def row2taxonomy(row):
    parts = []
    for rank_synonyms in TAXONOMY_RANKS:
        for col in rank_synonyms:
            col_ = col.lower()
            if col in row.keys() and row[col] and row[col] != "-":
                parts.append(row[col])
                break
            elif col_ in row.keys() and row[col_] and row[col_] != "-":
                parts.append(row[col_])
                break
        else:
            break
    return " > ".join(parts)

In [ ]:
def check_title(title):
    """
    Screen dataset title.

    Parameters
    ----------
    title : str
        The title of the dataset.

    Returns
    -------
    bool
        Whether the dataset title is acceptable.
    """

    title = str(title)

    if title.startswith("Meteorological observations"):
        return False
    if title.startswith("Sea ice conditions"):
        return False
    if "topsoil" in title.lower():
        return False
    if "core" in title.lower():
        # return False
        pass

    return True


def add_file_extension(row):
    """
    Add file extension to image filename.

    Parameters
    ----------
    row : dict
        A dict record which may have fields ``"image"``, ``"File format"``, ``"File type"``.

    Returns
    -------
    fname : str
        File name with extension included.
    """
    if (
        "image" not in row.keys()
        or not row["image"]
        or not isinstance(row["image"], str)
    ):
        return ""

    s = row["image"]
    ext = os.path.splitext(s)[-1]
    if (
        ext.lower()
        in checker.VALID_IMG_EXTENSIONS
        + checker.INVALID_FILE_EXTENSIONS
        + checker.COMPRESSED_FILE_EXTENSIONS
    ):
        return s

    for col in ["File format", "File type"]:
        if col not in row.keys():
            continue
        new_ext = row[col]
        if not new_ext or not isinstance(new_ext, str):
            continue
        new_ext = "." + new_ext.strip().lstrip(".")
        if ext == new_ext:
            break
        s += new_ext
        break

    return s


def datetime2timestamp(ts):
    """
    Convert a datetime string to a timestamp.

    Parameters
    ----------
    ts : str
        Datetime string, in a format understood by ``dateutil.parser``.

    Returns
    -------
    float
        Timestamp; number of seconds since Unix epoch.
    """
    if isinstance(ts, str):
        return dateutil.parser.parse(ts).timestamp()
    return ts


def reformat_df(df, remove_duplicate_columns=True):
    """
    Reformat/clean pangaea dataset.

    Rename columns.

    Parameters
    ----------
    df : pandas.DataFrame
        Original dataset.

    Returns
    -------
    df : pandas.DataFrame or None
        Cleaned dataset, or ``None`` if the dataset is invalid.
    """

    if (
        "dataset_title" in df
        and len(df) > 0
        and df.iloc[0]["dataset_title"]
        and not check_title(df.iloc[0]["dataset_title"])
    ):
        return None

    # Make a copy of the dataframe so we can't overwrite the input
    df = df.copy()

    # Remove bad columns
    df.drop(labels=["-"], axis="columns", inplace=True, errors="ignore")
    # Remove duplicately named columns
    cols_to_drop = []
    if remove_duplicate_columns:
        for col in df.columns:
            if len(col) < 2:
                continue
            if (
                (col[-2] in " _")
                and (col[-1] in "123456789")
                and (col[:-2] in df.columns)
            ):
                cols_to_drop.append(col)
        df.drop(labels=cols_to_drop, axis="columns", inplace=True)

    # Find the correct URL column, and drop other columns containing "url"
    cols_to_drop = []
    mapping = {}
    col_url = find_url_column(df)
    mapping[col_url] = "url"
    for col in df.columns:
        if col != col_url and "url" in col.lower():
            cols_to_drop.append(col)

    # Search for matches to canonical columns.
    # Each entry in desired_columns is a key, value pair where the key
    # is the output column name, and the value is a list of search names
    # in order of priority. The first match will be kept and others discarded.
    desired_columns = {
        "dataset": ["ds_id", "dataset", "Campaign", "campaign"],
        "site": ["Event", "event", "Site", "site", "deployment"],
        "image": ["image", "filename"],
        "datetime": [
            "Date/Time",
            "datetime",
            "timestamp",
            "date/timestart",
            "date/timeend",
            "date",
        ],
        "latitude": [
            "Latitude",
            "latitude",
            "lat",
            "latitude+",
            "latitudemed",
            "latitudenorth",
            "latitudesouth",
        ],
        "longitude": [
            "Longitude",
            "longitude",
            "lon",
            "long",
            "longitude+",
            "longitudemed",
            "longitudewest",
            "longitudeeast",
        ],
        "x_pos": [],
        "y_pos": [],
        "altitude": ["altitude", "height"],
        "depth": [
            "depthwater",
            "bathydepth",
            "bathymetry",
            "bathy",
            "depth",
            "elevation",
        ],
        "backscatter": [],
        "temperature": ["temperature", "temp"],
        "salinity": ["salinity", "sal"],
        "chlorophyll": [],
        "acidity": ["pH"],
        "doi": ["DOI", "doi"],
    }
    # Remove non-alphanumeric padding characters, including spaces, from actual column names
    raw_cols = list(df.columns)
    clean_cols = [
        col.lower().replace(" ", "").replace("-", "").replace("_", "").replace(".", "")
        for col in df.columns
    ]
    # Map to lower case
    lower_cols = [col.lower() for col in clean_cols]

    # Search for matching column names
    for canon, searches in desired_columns.items():
        found = False

        # Check for case-sensitive, non-alphanumeric, match
        for search in searches:
            if search not in raw_cols:
                continue
            col = search
            if not found:
                found = True
                mapping[col] = canon
                if col != canon and canon in df.columns:
                    cols_to_drop.append(canon)
            elif col not in mapping and col not in cols_to_drop:
                cols_to_drop.append(col)

        # Check for case-sensitive match
        for search in searches:
            if search not in clean_cols:
                continue
            col = df.columns[clean_cols.index(search)]
            if not found:
                found = True
                mapping[col] = canon
                if col != canon and canon in df.columns:
                    cols_to_drop.append(canon)
            elif col not in mapping and col not in cols_to_drop:
                cols_to_drop.append(col)

        # Check for case-insensitive match
        for search in searches:
            if search.lower() not in lower_cols:
                continue
            col = df.columns[lower_cols.index(search.lower())]
            if not found:
                found = True
                mapping[col] = canon
                if col != canon and canon in df.columns:
                    cols_to_drop.append(canon)
            elif col not in mapping and col not in cols_to_drop:
                cols_to_drop.append(col)

    # Remove superfluous columns
    df.drop(labels=cols_to_drop, axis="columns", inplace=True)
    # Rename columns to canonical names
    df.rename(columns=mapping, inplace=True, errors="raise")

    # Add file extension to image
    df["image"] = df.apply(add_file_extension, axis=1)
    # if "timestamp" not in df.columns and "datetime" in df.columns:
    #     df["timestamp"] = df["datetime"].apply(datetime2timestamp)

    if any([c in clean_cols for c in ["Kingdom", "Phylum", "Genus"]]):
        df["taxonomy"] = df.apply(row2taxonomy, axis=1)
        df.drop(
            labels=[x for syn in TAXONOMY_RANKS for x in syn],
            axis="columns",
            inplace=True,
            errors="ignore",
        )

    cols_to_drop = [
        "File format",
        "File type",
        "File size",
        "Date/Time",
        "Date/time end",
    ]
    df.drop(labels=cols_to_drop, axis="columns", inplace=True, errors="ignore")

    return df

In [ ]:
check_title("Sea ice conditions at location")

## Load data with dataset cleaning functions applied

In [ ]:
column_count = defaultdict(lambda: 0)
column_examples = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0
dfs = []
dfs_fnames = []

for fname in tqdm(os.listdir(dirname)):
    ds_id = os.path.splitext(fname)[0]
    df = pd.read_csv(os.path.join(dirname, fname))
    n_total += 1
    if not checker.has_url_col(df):
        continue

    url_col = find_url_column(df)
    if not url_col:
        files_without_url.append(fname)
        continue

    df["ds_id"] = f"pangaea-{ds_id}"
    df = reformat_df(df)
    if df is None:
        continue
    url_col = "url"
    df = df[df[url_col] != ""]
    if len(df) == 0:
        continue

    n_valid += 1

    for col in df.columns:
        column_count[col] += 1
        column_examples[col].append(fname)

    if len(df) != len(df.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

    dfs.append(df)
    dfs_fnames.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) valid datasets")
print(f"Of which {len(files_with_repeat_urls)} have repeated URLs")
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Merge datasets together

In [ ]:
select_cols = {
    "dataset",
    "site",
    "url",
    "image",
    "datetime",
    "latitude",
    "longitude",
    "x_pos",
    "y_pos",
    "altitude",
    "depth",
    "backscatter",
    "temperature",
    "salinity",
    "chlorophyll",
    "acidity",
}

df_all = pd.concat([df[df.columns.intersection(select_cols)] for df in dfs])

In [ ]:
df_all

In [ ]:
# Save all records
df_all.to_csv(
    f"../pangaea_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv", index=False
)

In [ ]:
# Filter down to only valid URLs
df_all = df_all[df_all["url"].apply(checker.is_url)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:4])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
url_base = unique_url_bases[0]
df_all[df_all["url"].str.startswith(url_base)].iloc[[0, -1]]

### Find bad subdomains

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
# Check how many rows have lat & lon
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
# Remove duplicate URLs
df_all = df_all.drop_duplicates(subset="url")

In [ ]:
len(df_all)

In [ ]:
# Check how many rows have lat & lon
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
is_image = df_all["url"].apply(
    lambda x: checker.has_img_extension(x.rstrip("/"))
) | df_all["image"].apply(lambda x: checker.has_img_extension(x.rstrip("/")))
df_all = df_all[is_image]

In [ ]:
len(df_all)

In [ ]:
sum(~df_all["latitude"].isna() & ~df_all["longitude"].isna())

In [ ]:
len(df_all["site"].unique())

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:4])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i + 1, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
def check_subdomain(url):
    blacklist = [
        "https://doi.org/10.1594/PANGAEA",
        "http://epic.awi.de/",
        "https://epic.awi.de/",
        "http://hdl.handle.net/10013/",
        "http://library.ucsd.edu/dc/object/",
        "https://hs.pangaea.de/Maps/",
        "https://hs.pangaea.de/Movies/",
        "https://hs.pangaea.de/Projects/",
        "https://hs.pangaea.de/bathy/",
        "https://hs.pangaea.de/fishsounder/",
        "https://hs.pangaea.de/mag/",
        "https://hs.pangaea.de/model/",
        "https://hs.pangaea.de/nav/",
        "https://hs.pangaea.de/palaoa/",
        "https://hs.pangaea.de/para/",
        "https://hs.pangaea.de/reflec/",
        "https://hs.pangaea.de/sat/",
        "https://prr.osu.edu/collection/object/",
        "https://store.pangaea.de/Projects/",  # Not all bad, but mostly
        "https://store.pangaea.de/Publications/",  # Not all bad, but mostly
        "https://store.pangaea.de/software/",
        "https://www.ngdc.noaa.gov/geosamples/",
    ]
    for entry in blacklist:
        if url.startswith(entry):
            return False
    return True

In [ ]:
# Remove bad URLs based on their subdomain
df_all = df_all[df_all["url"].apply(check_subdomain)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:5])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i + 1, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
import re

In [ ]:
if re.search(
    "(?<![A-Za-z])map(?![A-Za-z])",
    "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/config/divemap/Dive360_map.jpg",
):
    print("true")

In [ ]:
if re.search(
    "(?<![A-Za-z])map(?![A-Za-z])",
    "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/config/divemap/Dive360mapy.jpg",
):
    print("true")

In [ ]:
def check_subdomain(url):
    blacklist = [
        "https://doi.org/10.1594/PANGAEA",
        "http://epic.awi.de/",
        "https://epic.awi.de/",
        "http://hdl.handle.net/10013/",
        "http://library.ucsd.edu/dc/object/",
        "https://app.geosamples.org/uploads/UHM",
        "https://hs.pangaea.de/Images/Linescan",
        "https://hs.pangaea.de/Maps/",
        "https://hs.pangaea.de//Maps",
        "https://hs.pangaea.de/Movies/",
        "https://hs.pangaea.de/Projects/",
        "https://hs.pangaea.de/bathy/",
        "https://hs.pangaea.de/fishsounder/",
        "https://hs.pangaea.de/mag/",
        "https://hs.pangaea.de/model/",
        "https://hs.pangaea.de/nav/",
        "https://hs.pangaea.de/palaoa/",
        "https://hs.pangaea.de/pasata/",
        "https://hs.pangaea.de/para/",
        "https://hs.pangaea.de/polar",
        "https://hs.pangaea.de/reflec/",
        "https://hs.pangaea.de/sat/",
        "https://prr.osu.edu/collection/object/",
        "https://store.pangaea.de/Projects/",  # Not all bad, but mostly
        "https://store.pangaea.de/Publications/",  # Not all bad, but mostly
        "https://store.pangaea.de/software/",
        "https://www.ngdc.noaa.gov/geosamples/",
        "https://hs.pangaea.de/Images/Airphoto",
        # "https://hs.pangaea.de/Images/Cores",  # Some of these are okay
        "https://hs.pangaea.de/Images/Documentation",
        "https://hs.pangaea.de/Images/Maps",
        "https://hs.pangaea.de/Images/MMT/",
        "https://hs.pangaea.de/Images/Plankton",
        # The GeoB19346-1 dataset contains .bmp images of the ROV's sonar
        "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/data/sonar/",
        "https://hs.pangaea.de/Images/Satellite",
        "https://hs.pangaea.de/Images/SeaIce",
        "https://hs.pangaea.de/Images/Water",
        "https://store.pangaea.de/Images/Airphoto",
        "https://store.pangaea.de/Images/Documentation",
    ]
    banned_words = ["map", "divemap", "dredge_photos", "dredgephotograph"]
    for entry in blacklist:
        if url.startswith(entry):
            return False
    for word in banned_words:
        if re.search("(?<![A-Za-z])" + word + "(?![A-Za-z])", url.lower()):
            return False
    if re.search("(?<![a-z])core(?![a-rty])", url.lower()) and "SUR" not in url:
        # Images of cores must contain "SURFACE", or the shorthand "SUR"
        # We only keep the ones with surface in uppercase, because those
        # experiments are in-situ surface photos, whereas lower case are not.
        return False
    if "not_available" in url:
        return False
    return True

In [ ]:
# Remove bad URLs based on their subdomain (again), and remove maps
df_all = df_all[df_all["url"].apply(check_subdomain)]

In [ ]:
len(df_all)

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:5])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i + 1, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[499])
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[4999])
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
df_all[df_all["url"].apply(lambda x: "mosaic" in x)]

In [ ]:
# Drop mosaic images
df_all = df_all[~df_all["url"].apply(lambda x: "mosaic" in x)]

In [ ]:
unique_extensions = sorted(
    df_all["url"].apply(lambda x: os.path.splitext(x)[1]).unique()
)

In [ ]:
unique_extensions

In [ ]:
for i, ext in enumerate(unique_extensions):
    print()
    sdf = df_all[df_all["url"].str.endswith(ext)]
    print(
        "{:3d}/{} ({:7d} URLs), extension {}".format(
            i + 1, len(unique_extensions), len(sdf), ext
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[499])
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[4999])
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

In [ ]:
df_all

### Save unlabelled dataset

In [ ]:
df_all.to_csv(
    f"../pangaea_{datetime.datetime.today().strftime('%Y-%m-%d')}_filtered.csv",
    index=False,
)

### Check images for each title

In [ ]:
_dfs = []
_dfs_fnames = []

for df, df_fname in tqdm(zip(dfs, dfs_fnames), total=len(dfs)):
    url_column = "url"

    # Filter down to only valid URLs
    df = df[df[url_column].apply(checker.is_url)]
    if df is None or len(df) == 0:
        continue

    # Remove bad subdomains
    df = df[df[url_column].apply(check_subdomain)]
    if df is None or len(df) == 0:
        continue

    # Filter down to only rows which have image extension
    is_image = df[url_column].apply(lambda x: checker.has_img_extension(x.rstrip("/")))
    if "image" in df.columns:
        is_image |= df["image"].apply(
            lambda x: checker.has_img_extension(x.rstrip("/"))
        )
    df = df[is_image]
    if df is None or len(df) == 0:
        continue

    _dfs.append(df)
    _dfs_fnames.append(df_fname)

In [ ]:
print(f"Keeping {len(_dfs)} of {len(dfs)} datasets")

In [ ]:
dfs = _dfs
dfs_fnames = _dfs_fnames

In [ ]:
starter_length = 40
title_starters_to_row = {}

for df, fname in zip(dfs, dfs_fnames):
    if "dataset_title" not in df.columns:
        continue
    for title in df["dataset_title"].unique():
        if pd.isna(title):
            continue
        if title[:starter_length] in title_starters_to_row:
            continue
        subdf = df[df["dataset_title"] == title]
        title_starters_to_row[title[:starter_length]] = subdf.iloc[len(subdf) // 2]

In [ ]:
len(title_starters_to_row)

In [ ]:
for k in sorted(title_starters_to_row.keys()):
    print(f"{k:40} ... {title_starters_to_row[k]['url']}")

In [ ]:
for k in sorted(title_starters_to_row.keys()):
    print(title_starters_to_row[k]["dataset_title"])

In [ ]:
unique_url_bases = sorted(
    df_all["url"].apply(lambda x: "/".join(x.split("/")[:6])).unique()
)

In [ ]:
len(unique_url_bases)

In [ ]:
unique_url_bases = [
    x
    for x in unique_url_bases
    if x.startswith("https://hs.pangaea.de/bathy")
    or x.startswith("https://hs.pangaea.de/Images")
    or x.startswith("https://store.pangaea.de/Images")
]

In [ ]:
len(unique_url_bases)

In [ ]:
unique_url_bases

for i, url_base in enumerate(unique_url_bases):
    print()
    sdf = df_all[df_all["url"].str.startswith(url_base)]
    print(
        "{:3d}/{} ({:7d} URLs), base {}".format(
            i + 1, len(unique_url_bases), len(sdf), url_base
        )
    )
    print(sdf["url"].iloc[0])
    if len(sdf) > 2:
        print(sdf["url"].iloc[1])
    if len(sdf) > 4:
        print(sdf["url"].iloc[len(sdf) // 2])
    if len(sdf) > 12:
        print(sdf["url"].iloc[9])
    if len(sdf) > 102:
        print(sdf["url"].iloc[99])
    if len(sdf) > 1002:
        print(sdf["url"].iloc[499])
        print(sdf["url"].iloc[999])
    if len(sdf) > 10002:
        print(sdf["url"].iloc[4999])
        print(sdf["url"].iloc[9999])
    if len(sdf) > 3:
        print(sdf["url"].iloc[-2])
    if len(sdf) > 1:
        print(sdf["url"].iloc[-1])

#### Refine title and subdomain screening

In [ ]:
def check_title(title):
    """
    Screen dataset title.

    Parameters
    ----------
    title : str
        The title of the dataset.

    Returns
    -------
    bool
        Whether the dataset title is acceptable.
    """

    title = str(title)

    if "do not use" in title.lower():
        return False
    if title.startswith("Meteorological observations"):
        return False
    if title.startswith("Sea ice conditions"):
        return False
    if "topsoil" in title.lower():
        return False
    if "core" in title.lower():
        # return False
        pass
    if "aquarium" in title.lower():
        return False
    if " of the early life history " in title.lower():
        return False
    if "grab sample" in title.lower():
        return False
    if title.startswith("Calyx growth"):
        return False
    if "dried glass sponges" in title.lower():
        return False
    if "fresh glass sponges" in title.lower():
        return False
    if "spicule preparations" in title.lower():
        return False
    if title.startswith("Shell growth increments"):
        return False
    if title.startswith("Images of shell cross sections"):
        return False

    return True

In [ ]:
check_title("Grab samples from location")  # Test

In [ ]:
def check_subdomain(url):
    blacklist = [
        "https://doi.org/10.1594/PANGAEA",
        "http://epic.awi.de/",
        "https://epic.awi.de/",
        "http://hdl.handle.net/10013/",
        "http://library.ucsd.edu/dc/object/",
        "https://app.geosamples.org/uploads/UHM/",
        "https://hs.pangaea.de/Images/Linescan/",
        "https://hs.pangaea.de/Maps/",
        "https://hs.pangaea.de//Maps",
        "https://hs.pangaea.de/Movies/",
        "https://hs.pangaea.de/Projects/",
        "https://hs.pangaea.de/bathy/",
        "https://hs.pangaea.de/fishsounder/",
        "https://hs.pangaea.de/mag/",
        "https://hs.pangaea.de/model/",
        "https://hs.pangaea.de/nav/",
        "https://hs.pangaea.de/palaoa/",
        "https://hs.pangaea.de/pasata/",
        "https://hs.pangaea.de/para/",
        "https://hs.pangaea.de/polar",
        "https://hs.pangaea.de/reflec/",
        "https://hs.pangaea.de/sat/",
        "https://prr.osu.edu/collection/object/",
        "https://store.pangaea.de/Projects/",  # Not all bad, but mostly
        "https://store.pangaea.de/Publications/",  # Not all bad, but mostly
        "https://store.pangaea.de/software/",
        "https://www.ngdc.noaa.gov/geosamples/",
        "https://hs.pangaea.de/Images/Airphoto/",
        # "https://hs.pangaea.de/Images/Cores/",  # Some of these are okay
        "https://hs.pangaea.de/Images/Documentation/",
        "https://hs.pangaea.de/Images/Maps/",
        "https://hs.pangaea.de/Images/MMT/",
        "https://hs.pangaea.de/Images/Plankton/",
        # The GeoB19346-1 dataset contains .bmp images of the ROV's sonar
        "https://hs.pangaea.de/Images/ROV/M/M114/GeoB19346-1/data_publish/data/sonar/",
        "https://hs.pangaea.de/Images/Satellite/",
        "https://hs.pangaea.de/Images/SeaIce/",
        "https://hs.pangaea.de/Images/Water/",
        "https://store.pangaea.de/Images/Airphoto/",
        "https://store.pangaea.de/Images/Documentation/",
        "https://hs.pangaea.de/Images/Benthos/AWI_experimental_aquarium_system/",
        # "https://hs.pangaea.de/Images/Benthos/AntGlassSponges",  # Only okay if it contains "AHEAD"
        "https://hs.pangaea.de/Images/Benthos/Kongsfjorden/MHerrmann/",  # Microscope images
        "https://hs.pangaea.de/Images/Benthos/Kongsfjorden/Brandal/",  # Cross-sections
    ]
    banned_words = [
        "aquarium",
        "map",
        "divemap",
        "dredgephotos",
        "dredge_photos",
        "dredgephotograph",
        "grabsample",
        "grab_sample",
    ]
    for entry in blacklist:
        if url.startswith(entry):
            return False
    for word in banned_words:
        if re.search("(?<![A-Za-z])" + word + "(?![A-Za-z])", url.lower()):
            return False
    if re.search("(?<![a-z])core(?![a-rty])", url.lower()) and "SUR" not in url:
        # Images of cores must contain "SURFACE", or the shorthand "SUR"
        # We only keep the ones with surface in uppercase, because those
        # experiments are in-situ surface photos, whereas lower case are not.
        return False
    if (
        url.startswith("https://hs.pangaea.de/Images/Benthos/AntGlassSponges/")
        and "AHEAD" not in url
    ):
        # Images of AntGlassSponges must contain "AHEAD" to be kept
        # otherwise, they are of sponges after removal
        return False
    if "not_available" in url:
        return False
    return True

## Load data with URL filtering functions

In [ ]:
def filter_urls(df, url_column="url", inplace=True):
    if not inplace:
        df = df.copy()

    if df is None or len(df) == 0:
        return df

    # Filter down to only valid URLs
    df = df[df[url_column].apply(checker.is_url)]
    if df is None or len(df) == 0:
        return df

    # Remove bad subdomains
    df = df[df[url_column].apply(check_subdomain)]
    if df is None or len(df) == 0:
        return df

    # Filter down to only rows which have image extension
    is_image = df[url_column].apply(lambda x: checker.has_img_extension(x.rstrip("/")))
    if "image" in df.columns:
        is_image |= df["image"].apply(
            lambda x: checker.has_img_extension(x.rstrip("/"))
        )
    df = df[is_image]
    if df is None or len(df) == 0:
        return df

    # Drop mosaic images
    df = df[~df[url_column].apply(lambda x: "mosaic" in x.lower())]

    return df

In [ ]:
def fixup_url_with_image(row):
    url_old = row["url"]
    if "image" not in row or not row["image"]:
        return url_old
    url = url_old.rstrip("/ ")
    url_parts = url.split("/")
    ext = os.path.splitext(url_parts[-1])[-1]
    url_new = "/".join(url_parts[:-1])
    if row["image"] + ext == url_parts[-1]:
        return url
    url_new += "/" + row["image"]
    return url_new


def insert_rows(df, rows, indices):
    parts = [df[: indices[0]], pd.DataFrame([rows[0]])]
    for j in range(len(indices) - 1):
        parts.append(df[indices[j - 1] : indices[j]])
        parts.append(pd.DataFrame([rows[j]]))
    parts.append(df[indices[-1] :])
    return pd.concat(parts)


def fixup_repeated_urls(
    df, url_column="url", inplace=True, force_keep_original=True, verbose=1
):
    if not inplace:
        df = df.copy()
    dup_urls = df[df.duplicated(subset=url_column)][url_column].unique()
    if "image" not in df.columns:
        if len(dup_urls) and verbose >= 1:
            print(
                f"Can't clean {len(dup_urls)} repeated URLs in {df.loc[0, 'ds_id']} without 'image' column"
            )
        return df
    if len(dup_urls) == 0:
        return df
    if verbose >= 1 and "dataset" in df.columns:
        print(f"{df.iloc[0]['dataset']} has {len(dup_urls)} duplicated URLs")
    rows_to_insert = []
    indices_to_insert_at = []
    for dup_url in dup_urls:
        if pd.isna(dup_url):
            continue
        is_bad_url = df[url_column] == dup_url
        if verbose >= 2:
            n_to_change = sum(is_bad_url)
            print(f"Fixing up {n_to_change} repetitions of the URL {dup_url}")
        first_row_idx = np.nonzero(is_bad_url.values)[0][0]
        if force_keep_original:
            # first_row = df[is_bad_url].iloc[0].copy()
            first_row = df.iloc[first_row_idx].copy()
        df.loc[is_bad_url, url_column] = df[is_bad_url].apply(
            fixup_url_with_image, axis=1
        )
        n_remain = sum(df.loc[is_bad_url, url_column] == dup_url)
        if verbose >= 2:
            if n_remain == n_to_change:
                print(f"  All {n_to_change} URLs left unchanged")
            else:
                print(f"  {n_remain} / {n_to_change} URLs remain unchanged")
                if verbose >= 3:
                    print("  After:")
                    for x in df.loc[is_bad_url, url_column][:5]:
                        print(f"    {x}")
                    if n_to_change > 5:
                        print("    ...")
                        print("    " + df.loc[is_bad_url, url_column].values[-1])

        if force_keep_original and n_remain == 0:
            if verbose >= 1:
                print(f"  Duplicating a row since the URL {dup_url} no longer appears")
            first_row[url_column] = dup_url
            rows_to_insert.append(first_row)
            indices_to_insert_at.append(first_row_idx)
    if len(rows_to_insert) > 0:
        if verbose >= 1:
            print(f"  Inserting {len(rows_to_insert)} duplicated rows")
        df = insert_rows(df, rows_to_insert, indices_to_insert_at)
    return df

In [ ]:
column_count = defaultdict(lambda: 0)
column_examples = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
files_with_repeat_urls2 = []
n_total = 0
n_valid = 0
dfs = []
dfs_fnames = []

for fname in tqdm(sorted(sorted(os.listdir(dirname)), key=len)):
    # for fname in tqdm(os.listdir(dirname)):
    ds_id = os.path.splitext(fname)[0]
    df = pd.read_csv(os.path.join(dirname, fname))
    n_total += 1
    if not checker.has_url_col(df):
        continue

    url_col = find_url_column(df)
    if not url_col:
        files_without_url.append(fname)
        continue

    df["ds_id"] = f"pangaea-{ds_id}"
    df = reformat_df(df)
    if df is None:
        continue

    url_col = "url"
    df = df[df[url_col] != ""]
    if len(df) == 0:
        continue

    df = filter_urls(df, url_column=url_col)
    if len(df) == 0:
        continue

    n_valid += 1

    for col in df.columns:
        column_count[col] += 1
        column_examples[col].append(fname)

    # Drop rows that are complete duplicates
    df.drop_duplicates(inplace=True)

    if len(df) != len(df.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

    # Try to fix repeated URLs that are accidental dups but should differ
    df = fixup_repeated_urls(df, url_column=url_col, verbose=1)

    if len(df) != len(df.drop_duplicates(subset=url_col)):
        files_with_repeat_urls2.append(fname)

    # Check for any rows that are all NaNs
    if sum(df.isna().all("columns")) > 0:
        print(f"{ds_id} has a row which is all NaNs")

    dfs.append(df)
    dfs_fnames.append(fname)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) valid datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (before replacing dups with image)"
)
print(
    f"Of which {len(files_with_repeat_urls2)} have repeated URLs (after replacing dups with image)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

In [ ]:
select_cols = {
    "dataset",
    "site",
    "url",
    "image",
    "datetime",
    "latitude",
    "longitude",
    "altitude",
    "depth",
    "backscatter",
    "temperature",
    "salinity",
    "chlorophyll",
    "acidity",
}

df_all = pd.concat(
    [df[df.columns.intersection(select_cols)] for df in dfs if len(df) > 0]
)

In [ ]:
len(df_all)

In [ ]:
# Remove duplicate URLs
df_all.drop_duplicates(subset="url", inplace=True, keep="first")

In [ ]:
df_all

In [ ]:
df_all[~df_all["url"].apply(checker.is_url)]

In [ ]:
df_all[df_all.isna().all("columns")]

In [ ]:
unique_extensions = sorted(
    df_all["url"].apply(lambda x: os.path.splitext(x)[1]).unique()
)
unique_extensions

In [ ]:
df_all[
    df_all["url"]
    == "https://hs.pangaea.de/Images/Benthos/Great_Barrier_Reef/CCMR/Opal_Reef_2017-01/20170126_Opal_S_C14_289.jpg"
]

In [ ]:
df_all[
    df_all["url"]
    == "https://hs.pangaea.de/Images/Benthos/Great_Barrier_Reef/CCMR/Opal_Reef_2017-01/20170126_Opal_DC2-002.jpg"
]

### Fix repeated output paths

In [ ]:
def row2basename(row):
    basename = row["image"]
    if pd.isna(basename) or not basename:
        if pd.isna(row["url"]):
            return ""
        basename = row["url"].rstrip("/").split("/")[-1]
    basename = sanitize_filename(basename)
    ext = os.path.splitext(basename)[1]
    expected_ext = os.path.splitext(row["url"].rstrip("/"))[1]
    if expected_ext and ext.lower() != expected_ext.lower():
        if ext.lower() in {".jpg", ".jpeg"}:
            basename = os.path.splitext(basename)[0] + expected_ext
        else:
            basename = basename + expected_ext
    return basename


def determine_outpath(df):
    return (
        sanitize_filename_series(df["dataset"])
        + "/"
        + sanitize_filename_series(df["site"])
        + "/"
        + df.apply(row2basename, axis=1)
    )


def fixup_repeated_output_paths(df, inplace=True, verbose=1):
    if not inplace:
        df = df.copy()
    df["_outpath"] = determine_outpath(df)
    dup_outpaths = df[df.duplicated(subset="_outpath")]["_outpath"].unique()

    if len(dup_outpaths) == 0:
        return df

    if verbose >= 1:
        print(
            f"There are {len(dup_outpaths)} duplicated output paths in this dataframe"
        )

    for dup_outpath in dup_outpaths:
        is_bad = df["_outpath"] == dup_outpath
        n_bad = sum(is_bad)

        if verbose >= 2:
            print(f"Trying to fix up {n_bad} repetitions of the path {dup_outpath}")

        # 1. Try taking the basename from the URL instead
        # We will try taking just the last bit (photoname.jpg), then including preceding bits
        # of the URL (subsite/photoname.jpg, site/subsite/photoname.jpg)
        subdf = df[is_bad]
        resolved = False
        for k in range(1, 5):
            new_basenames = subdf["url"].apply(
                lambda x: sanitize_filename("-".join(x.rstrip("/").split("/")[-k:]))
            )
            # All URL basenames are unique, so we are done
            if len(new_basenames.unique()) != len(new_basenames):
                continue
            df.loc[is_bad, "image"] = new_basenames
            if verbose >= 2:
                print(f"  Using last {k} part(s) of the URL as the basename")
            resolved = True
            break
        if resolved:
            continue

        # 2. If that didn't work, just append _0, _1, ... _N to the image names
        if verbose >= 2:
            print("  Appending a suffix to the basename to prevent collisions")
        new_basenames = subdf.apply(row2basename, axis=1)
        new_basenames = (
            new_basenames.apply(lambda x: os.path.splitext(x)[0])
            + "_"
            + pd.Series([str(x) for x in range(n_bad)], index=new_basenames.index)
            + new_basenames.apply(lambda x: os.path.splitext(x)[1])
        )
        df.loc[is_bad, "image"] = new_basenames

    df.drop(columns="_outpath", inplace=True)
    return df

In [ ]:
df_all = fixup_repeated_output_paths(df_all, inplace=True, verbose=2)

In [ ]:
df_all

### Save unlabelled dataset

In [ ]:
df_all.to_csv(
    f"../pangaea_{datetime.datetime.today().strftime('%Y-%m-%d')}_filtered_no-repeats_sorted-first.csv",
    index=False,
)

# Labelled data

## Percent Coverage

In [ ]:
cov_dfs = [dfs[dfs_fnames.index(fname)] for fname in coverage_datasets]

In [ ]:
df_cov_all = pd.concat(cov_dfs)

In [ ]:
display(df_cov_all)

In [ ]:
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

dois = []

for df in cov_dfs:
    n_total += 1
    url_col = "url"
    if not url_col:
        print(f"Missing url column with columns\n{df.columns}")
        continue
    n_valid += 1
    dois.append(df.iloc[0]["doi"])
    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)

for doi in sorted(dois):
    print(doi)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    if " cov" not in col:
        pass
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

### Labelled data coverage columns (A) PANGAEA > Roelfsema et. al.

Formed from the following 4 dataset "publication series":

    https://doi.pangaea.de/10.1594/PANGAEA.891711 (CC-BY-3.0)
    https://doi.pangaea.de/10.1594/PANGAEA.891736 (CC-BY-3.0)
    https://doi.pangaea.de/10.1594/PANGAEA.892623 (CC-BY-3.0)
    https://doi.pangaea.de/10.1594/PANGAEA.894801 (CC-BY-4.0)

All datasets have the same 61 coverage column labels:

    Acropora cov
    Acropora cov_2
    Acroporidae cov
    Acroporidae cov_2
    Acroporidae cov_3
    Acroporidae cov_4
    Montipora cov
    Montipora cov_2
    Acropora cov_3
    Acropora cov_4
    Acropora cov_5
    Acropora cov_6
    Favia cov
    Favia cov_2
    Coral indet cov
    Coral indet cov_2
    Coral indet cov_3
    Coral indet cov_4
    Pocilloporidae cov
    Pocilloporidae cov_2
    P. cylindrica cov
    P. cylindrica cov_2
    P. lichen cov
    P. lichen cov_2
    P. lobata cov
    P. lobata cov_2
    Coral indet cov_5
    Coral indet cov_6
    Coral indet cov_7
    Coral indet cov_8
    Alcyoniidae cov
    Alcyoniidae cov_2
    Gorgonia cov
    Gorgonia cov_2
    A. planci cov
    Invertebrata cov
    Soft corals oth cov
    Soft corals oth cov_2
    Invertebrata cov_2
    Sand cov
    Other cov
    Other cov_2
    Other cov_3
    Other cov_4
    Background cov
    Algae cov
    Benth microalgae cov
    Corall algae cov
    Corall algae cov_2
    Caulerpa sp. cov
    Chlorodesmis sp. cov
    Cyanobact cov
    Dictyota sp. cov
    Epith algal matrix cov
    Epith algal matrix cov_2
    Lobophora cov
    Halimeda sp. cov
    Sargassum sp. cov
    Padina sp. cov
    Turbinaria sp. cov
    Seagr cov

And all have metadata columns:

    image
    url
    latitude
    longitude
    Cover branch cor
    Cover branch cor_2
    site
    dataset_title
    doi
    dataset
    timestamp

Note that not all of the labels are manually generated. Abstract for one of them:
A subset of photoquadrats were uploaded to the CoralNet machine learning interface (https://coralnet.ucsd.edu/) and manually labelled for coral, algae or substrate type using 50 points per quadrat. Follow training of the machine, this enabled automatic annotation of all unclassified field images: the remaining field photos were uploaded to the database and 50 annotation points were overlaid on each of the images. Every point was assigned a benthic cover category from a label list automatically by the program. The resulting benthic cover data of each photo was linked to gps coordinates, saved as an ArcMap point shapefile, and projected to Universal Transverse Mercator WGS84 Zone 55 South.

In [ ]:
column_count_ = defaultdict(lambda: 0)
column_examples_ = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

dois = []

first = True

cov_dfs_sub = []

for df in cov_dfs:
    if "seagr cov" not in [c.lower() for c in df.columns]:
        continue
    if "acropora cov" not in [c.lower() for c in df.columns]:
        continue
    n_total += 1
    url_col = "url"
    if not url_col:
        print(f"Missing url column with columns\n{df.columns}")
        continue
    n_valid += 1
    dois.append(df.iloc[0]["doi"])
    cov_dfs_sub.append(df)

    for col in df.columns:
        column_count_[col] += 1
        column_examples_[col].append(fname)

    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)
    if first:
        for c in df.columns:
            if " cov" not in c:
                print(c)
        for c in df.columns:
            if " cov" in c:
                print(c)
        first = False
    last_df = df

for doi in sorted(dois):
    print(doi)

for c in last_df.columns:
    if " cov" not in c:
        print(c)
for c in last_df.columns:
    if " cov" in c:
        print(c)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count_)} unique column names:")
print()

for col, count in dict(
    sorted(column_count_.items(), key=lambda item: item[1], reverse=True)
).items():
    if " cov" not in col:
        pass
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

In [ ]:
label_columns = [k for k in column_count_ if " cov" in k]

cov_df_sub = pd.concat(cov_dfs_sub)

print(f"Number of records: {len(cov_df_sub):6d}  (initial)")

cov_df_sub = cov_df_sub[~cov_df_sub[label_columns].isna().all("columns")]

print(
    f"Number of records: {len(cov_df_sub):6d}  (remove records where all label columns are NaN)"
)

cols_for_dup = list(cov_df_sub.columns)
cols_for_dup.remove("dataset")
cols_for_dup.remove("dataset_title")
cols_for_dup.remove("doi")
cols_for_dup.remove("site")
cols_for_dup.remove("image")
# cols_for_dup.remove("Method comm")
cov_df_sub.drop_duplicates(subset=cols_for_dup, inplace=True)

print(
    f"Number of records: {len(cov_df_sub):6d}  (after removing duplicates across datasets)"
)
print(f"Number of uq urls: {len(cov_df_sub.drop_duplicates(subset='url')):6d}")

In [ ]:
cov_df_sub[cov_df_sub.duplicated(subset="url")]["dataset"].unique()

In [ ]:
cov_df_sub[cov_df_sub.duplicated(subset="url")]

In [ ]:
dup_urls = cov_df_sub[cov_df_sub.duplicated(subset="url")]["url"].unique()

for url in dup_urls:
    print(url, sum(cov_df_sub["url"] == url))

We found a URL, repeated 1982 times, which was not correct. The correct URLs are as per the image field:

- URL: https://hs.pangaea.de/Images/Benthos/Great_Barrier_Reef/CCMR/Opal_Reef_2017-01/20170126_Opal_S_C14_289.jpg
- Image: https://hs.pangaea.de/Images/Benthos/Great_Barrier_Reef/CCMR/Opal_Reef_2017-01/20170126_Opal_DC2-002.jpg

This is now fixed by the ``fixup_repeated_urls`` function.

We also found one URL which is repeated incorrectly again. The correct URLs are as per the image field:

- 1: https://hs.pangaea.de/Images/Benthos/Heron_Reef/2018/20181113_Heron_HeronBommie_-001.jpg
- 2: https://hs.pangaea.de/Images/Benthos/Heron_Reef/2018/20181114_Heron_HarrysBommie_-001.jpg

In [ ]:
cov_df_sub = fixup_repeated_urls(cov_df_sub)

In [ ]:
print(f"Number of records: {len(cov_df_sub):6d}")
print(f"Number of uq urls: {len(cov_df_sub.drop_duplicates(subset='url')):6d}")

cov_df_sub.to_csv(
    f"../pangaea_coverage-a_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv",
    index=False,
)
display(cov_df_sub)

### Labelled data coverage columns (B) PANGAEA > Roelfsema et. al.

All datasets in the publication series
https://doi.pangaea.de/10.1594/PANGAEA.846147 (CC-BY-3.0)

Sited at Eastern Banks, Moreton Bay.

Published in https://doi.org/10.1038/sdata.2015.40

Note
The 2015 data was additional data that was collected in the same manner as the other years. Although it was not used in the publication, we believed that it should be added to the data set as it expands the collection for anyone wanting to use the data in their own way.

In [ ]:
column_count_ = defaultdict(lambda: 0)
column_examples_ = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

dois = []

first = True

cov_dfs_sub = []

for df in cov_dfs:
    if "seagr cov" not in [c.lower() for c in df.columns]:
        continue
    if "acropora cov" in [c.lower() for c in df.columns]:
        continue
    n_total += 1
    url_col = "url"
    if not url_col:
        print(f"Missing url column with columns\n{df.columns}")
        continue
    n_valid += 1
    dois.append(df.iloc[0]["doi"])
    cov_dfs_sub.append(df)

    for col in df.columns:
        column_count_[col] += 1
        column_examples_[col].append(fname)

    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)
    if first:
        for c in df.columns:
            if " cov" not in c:
                print(c)
        for c in df.columns:
            if " cov" in c:
                print(c)
        first = False
    last_df = df

for doi in sorted(dois):
    print(doi)

for c in last_df.columns:
    if " cov" not in c:
        print(c)
for c in last_df.columns:
    if " cov" in c:
        print(c)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count_)} unique column names:")
print()

for col, count in dict(
    sorted(column_count_.items(), key=lambda item: item[1], reverse=True)
).items():
    if " cov" not in col:
        pass
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

In [ ]:
label_columns = [k for k in column_count_ if " cov" in k]

cov_df_sub = pd.concat(cov_dfs_sub)

print(f"Number of records: {len(cov_df_sub):6d}  (initial)")

cov_df_sub = cov_df_sub[~cov_df_sub[label_columns].isna().all("columns")]

print(
    f"Number of records: {len(cov_df_sub):6d}  (remove records where all label columns are NaN)"
)

cols_for_dup = list(cov_df_sub.columns)
cols_for_dup.remove("dataset")
cols_for_dup.remove("dataset_title")
cols_for_dup.remove("doi")
cols_for_dup.remove("site")
cols_for_dup.remove("image")
# cols_for_dup.remove("Method comm")
cov_df_sub.drop_duplicates(subset=cols_for_dup, inplace=True)

print(
    f"Number of records: {len(cov_df_sub):6d}  (after removing duplicates across datasets)"
)
print(f"Number of uq urls: {len(cov_df_sub.drop_duplicates(subset='url')):6d}")

In [ ]:
cov_df_sub[cov_df_sub.duplicated(subset="url")]

In [ ]:
cov_df_sub[cov_df_sub.duplicated(subset="url")].iloc[0]["url"]

In [ ]:
cov_df_sub[
    cov_df_sub["url"]
    == "https://store.pangaea.de/Images/Benthos/EasternBanks/2004-07/2004-07-30_AM3_transect_023.jpg"
]

In [ ]:
for col, vals in cov_df_sub[
    cov_df_sub["url"]
    == "https://store.pangaea.de/Images/Benthos/EasternBanks/2004-07/2004-07-30_AM3_transect_023.jpg"
].iteritems():
    if isinstance(vals.iloc[0], str):
        if (vals == vals.iloc[0]).all():
            continue
    else:
        if np.allclose(vals.iloc[0], vals, equal_nan=True):
            continue

    print(col)
    print(vals)
    print()

In [ ]:
# Drop this record, since labels are incompatible
cov_df_sub = cov_df_sub[
    cov_df_sub["url"]
    != "https://store.pangaea.de/Images/Benthos/EasternBanks/2004-07/2004-07-30_AM3_transect_023.jpg"
]

In [ ]:
print(f"Number of records: {len(cov_df_sub):6d}")
print(f"Number of uq urls: {len(cov_df_sub.drop_duplicates(subset='url')):6d}")

cov_df_sub.to_csv(
    f"../pangaea_coverage-b_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv",
    index=False,
)

display(cov_df_sub)

### Labelled data coverage columns (C) Other

The rest. 3 datasets in Antarctica, Arctic, and Chile.

    https://doi.pangaea.de/10.1594/PANGAEA.839225 (CC-BY-3.0)
    https://doi.pangaea.de/10.1594/PANGAEA.841459 (CC-BY-3.0)
    https://doi.pangaea.de/10.1594/PANGAEA.897047 (CC-BY-4.0)
    https://aslopubs.onlinelibrary.wiley.com/doi/10.1002/lno.11187

They are all interested in Bryozoa, but otherwise there isn't any overlap in labels.

In [ ]:
column_count_ = defaultdict(lambda: 0)
column_examples_ = defaultdict(lambda: [])
files_without_url = []
files_with_repeat_urls = []
n_total = 0
n_valid = 0

dois = []

first = True

cov_dfs_sub = []

for df in cov_dfs:
    if "seagr cov" in [c.lower() for c in df.columns]:
        continue
    if "acropora cov" not in [c.lower() for c in df.columns]:
        pass
    n_total += 1
    url_col = "url"
    if not url_col:
        print(f"Missing url column with columns\n{df.columns}")
        continue
    n_valid += 1
    dois.append(df.iloc[0]["doi"])
    print("{}: {}".format(df.iloc[0]["doi"], len(df)))
    cov_dfs_sub.append(df)

    for col in df.columns:
        column_count_[col] += 1
        column_examples_[col].append(fname)

    subdf = df[df[url_col] != ""]
    if len(subdf) != len(subdf.drop_duplicates(subset=url_col)):
        files_with_repeat_urls.append(fname)
    if first:
        for c in df.columns:
            if " cov" not in c:
                print(c)
        for c in df.columns:
            if " cov" in c:
                print(c)
        first = False
    last_df = df

for c in last_df.columns:
    if " cov" not in c:
        print(c)
for c in last_df.columns:
    if " cov" in c:
        print(c)

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) total datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count_)} unique column names:")
print()

for col, count in dict(
    sorted(column_count_.items(), key=lambda item: item[1], reverse=True)
).items():
    if " cov" not in col:
        pass
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

In [ ]:
for cov_df_sub in cov_dfs_sub:
    dataset = cov_df_sub.loc[0, "dataset"]
    print()
    print(dataset)
    print()

    label_columns = [k for k in cov_df_sub.columns if " cov" in k]

    print(f"Number of label columns: {len(label_columns)}")
    print(f"Number of records: {len(cov_df_sub):6d}  (initial)")

    cov_df_sub = cov_df_sub[~cov_df_sub[label_columns].isna().all("columns")]

    print(
        f"Number of records: {len(cov_df_sub):6d}  (remove records where all label columns are NaN)"
    )

    cols_for_dup = list(cov_df_sub.columns)
    cols_for_dup.remove("dataset")
    cols_for_dup.remove("dataset_title")
    cols_for_dup.remove("doi")
    cols_for_dup.remove("site")
    cols_for_dup.remove("image")
    # cols_for_dup.remove("Method comm")
    cov_df_sub = cov_df_sub.drop_duplicates(subset=cols_for_dup)

    print(
        f"Number of records: {len(cov_df_sub):6d}  (after removing duplicates across datasets)"
    )
    print(f"Number of uq urls: {len(cov_df_sub.drop_duplicates(subset='url')):6d}")

    if len(cov_df_sub) > 1000:
        dsid = dataset.replace("pangaea-", "")
        cov_df_sub.to_csv(
            f"../pangaea_coverage-{dsid}_{datetime.datetime.today().strftime('%Y-%m-%d')}.csv",
            index=False,
        )
    print()

## Additional labels

### Redo column name tally

But with URLs filtering

In [ ]:
print(f"There are {n_valid} valid (of {n_total}) valid datasets")
print(
    f"Of which {len(files_with_repeat_urls)} have repeated URLs (possibly multiple annotations)"
)
print()
print(f"There are {len(column_count)} unique column names:")
print()

for col, count in dict(
    sorted(column_count.items(), key=lambda item: item[1], reverse=True)
).items():
    if " cov" in col:
        continue
    c = col + " "
    print(f"{c:.<35s} {count:4d}")

In [ ]:
fname = column_examples["Marine litter"][0]
df = pd.read_csv(os.path.join(dirname, fname))
url_column = find_url_column(df)
print(url_column)
# Remove bad URLs
df = df[df[url_column].apply(checker.is_url)]
display(df)
print(df.columns)
print(df[url_column].iloc[0])

In [ ]:
df.describe()

In [ ]:
fname = column_examples["Plank"][0]
df = pd.read_csv(os.path.join(dirname, fname))
url_column = find_url_column(df)
print(url_column)
# Remove bad URLs
df = df[df[url_column].apply(checker.is_url)]
display(df)
print(df.columns)
print(df[url_column].iloc[0])

In [ ]:
fname = column_examples["Squat lobster"][0]
df = pd.read_csv(os.path.join(dirname, fname))
url_column = find_url_column(df)
print(url_column)
# Remove bad URLs
df = df[df[url_column].apply(checker.is_url)]
display(df)
print(df.columns)
print(df[url_column].iloc[0])

In [ ]:
fname = column_examples["Hard substrat"][0]
df = pd.read_csv(os.path.join(dirname, fname))
url_column = find_url_column(df)
print(url_column)
# Remove bad URLs
df = df[df[url_column].apply(checker.is_url)]
display(df)
print(df.columns)
print(df[url_column].iloc[0])
print(df["doi"].iloc[0])